In [10]:
using LinearMaps, IterativeSolvers, LinearAlgebra

# Dummy FLOP counters
global NFLOPs = 0
count_matmul_flops(args...) = nothing
count_vec_add_flops(args...) = nothing
count_vec_scaling_flops(args...) = nothing

# Your correction_equations_minres function here
function correction_equations_minres(A, U, lambdas, R; tol=1e-1, maxiter=100)
    global NFLOPs
    n, k = size(U)
    S = zeros(eltype(A), n, k)

    for j in 1:k
        λ, r = lambdas[j], R[:, j]

        M_apply = function(x)
            x_perp = x - (U * (U' * x)); 
            tmp = (A * x_perp) - λ * x_perp; 
            res = tmp - (U * (U' * tmp)); 
            return res
        end

        M_op = LinearMap{eltype(A)}(M_apply, n, n; ishermitian=true)

        rhs = r - (U * (U' * r)); 
        rhs = -rhs

        # Call MINRES with logging
        s_j, history = minres(M_op, rhs; reltol=tol, maxiter=maxiter, log=true)

        println("Vector $j: MINRES iterations = ", history.iters)

        s_j = s_j - (U * (U' * s_j)); 
        S[:, j] = s_j
    end
    return S
end

# === Example Problem ===
n = 10
A = Symmetric(randn(n, n))  # symmetric matrix
U = randn(n, 2)             # trial subspace
lambdas = [1.0, 2.0]        # fake Ritz values
R = randn(n, 2)             # fake residuals

S = correction_equations_minres(A, U, lambdas, R; tol=1e-6, maxiter=50)
println("Solution matrix S =")
println(S)


Vector 1: MINRES iterations = 10
Vector 2: MINRES iterations = 12
Solution matrix S =
[-0.9613579329309714 3.571707756537952; 0.700396557830945 -7.115493273068982; 0.522970216480914 -15.539018964339231; -1.0413618003208778 -0.5227140086030193; 0.004765061342248744 -11.242580273794966; 0.05505890406555064 1.0095838334364484; -0.04855446220646281 -10.195568937413569; -0.9357192225309595 6.675951483553281; -0.25722811883293417 4.805366216815828; -0.7255362463195205 9.203927904504875]


In [2]:
using BenchmarkTools

function algo1(n)
    return sum(1:n)
end

function algo2(n)
    return n * (n + 1) ÷ 2
end

@btime algo1(10^6)
@btime algo2(10^6)


  1.209 ns (0 allocations: 0 bytes)
  1.208 ns (0 allocations: 0 bytes)


500000500000

In [3]:
using BenchmarkTools, Plots

function benchmark_scaling(f, sizes)
    times = Float64[]
    for n in sizes
        t = @belapsed f($n)   # high-precision elapsed time
        push!(times, t)
    end
    return times
end

sizes = 10 .^ (2:6)
times1 = benchmark_scaling(algo1, sizes)
times2 = benchmark_scaling(algo2, sizes)

plot(sizes, times1, label="algo1", xscale=:log10, yscale=:log10, xlabel="Input size n", ylabel="Time (s)")
plot!(sizes, times2, label="algo2")


UndefVarError: UndefVarError: `f` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [4]:
Profile.clear()
@profile algo1(10^6)
using ProfileView   # nice GUI flamegraphs
ProfileView.view()


UndefVarError: UndefVarError: `Profile` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: Profile is loaded but not imported in the active module Main.

In [5]:
using PAPI

PAPI.start_counters(["PAPI_FP_OPS"]) do
    A = rand(1000,1000)
    B = rand(1000,1000)
    C = A * B
end


InitError: InitError: PAPI.PAPIError("Invalid argument")
during initialization of module PAPI

In [6]:
using IterativeSolvers
using LinearAlgebra

# Example system: A * x = b
A = [4.0 1.0;
     1.0 3.0]
b = [1.0, 2.0]

# Solve with GMRES
x, history = gmres(A, b; log=true)

println("Solution x = ", x)
println("Residual norms = ", history.data)

# exact solution
x_exact = A \ b
println("Exact solution x = ", x_exact)

Solution x = [0.09090909090909091, 0.6363636363636362]
Residual norms = Dict{Symbol, Any}(:reltol => 1.4901161193847656e-8, :abstol => 0.0, :resnorm => [0.5423261445466404, 1.0092936587501423e-16])
Exact solution x = [0.09090909090909091, 0.6363636363636364]
